## Get Today's Game
The next section uses the Odds API (a paid API with a somewhat generous free tier)

In [24]:
import requests
import datetime

# API URL
events_api_url = "https://api.the-odds-api.com/v4/sports/basketball_nba/events"

# Your API key
api_key = "a23c9884795b77de65081cc5fa65b8ff"


# Create parameters for the GET request
params = {
    "apiKey": api_key,
}

def get_today_nba_events():
    # Compute today's date in ISO format (YYYY-MM-DD)
    today_date = datetime.date.today().isoformat()

    # Send the GET request
    response = requests.get(events_api_url, params=params)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        games = response.json()  # Convert the response to JSON
        today_games = [game for game in games if game['commence_time'].startswith(today_date)]
        if not today_games:
            print("No games found for today.")
        return today_games
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
    return []

## Get Player Props

In [18]:
import requests

# API URL
event_odds_api_url = f"https://api.the-odds-api.com/v4/sports/basketball_nba/events/{event_id}/odds"

# Your API key
api_key = "a23c9884795b77de65081cc5fa65b8ff"

def get_player_props(event_id):
    # Parameters for the GET request
    params = {
        "apiKey": api_key,
        "regions": "us",
        "markets": "player_points",
        "oddsFormat": "american"
    }

    # Send the GET request
    response = requests.get(event_odds_api_url, params=params)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        data = response.json()
        bookmakers = data.get("bookmakers", [])
        
        # Find the "fanduel" bookmaker in the list of bookmakers
        for bookmaker in bookmakers:
            if bookmaker["key"] == "fanduel":
                fanduel_odds = []
                markets = bookmaker.get("markets", [])
                
                # Iterate through markets to find player_points
                for market in markets:
                    if market["key"] == "player_points":
                        outcomes = market.get("outcomes", [])
                        
                        # Iterate through outcomes to extract Fanduel odds
                        for outcome in outcomes:
                            if outcome.get("name") == "Over" or outcome.get("name") == "Under":
                                fanduel_odds.append({
                                    "name": outcome.get("description"),
                                    "price": outcome.get("price"),
                                    "point": outcome.get("point"),
                                    "type": outcome.get("name")
                                })
                
                return fanduel_odds
        return None  # If "fanduel" bookmaker not found
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
        return None

## Helpers


In [32]:
import datetime
import pytz

def get_date_formatted_pt(t):
    timestamp_utc_dt = datetime.datetime.fromisoformat(t.replace('Z', '+00:00'))
    # Define the Pacific Time (PT) timezone
    pt_timezone = pytz.timezone('US/Pacific')
    # Convert the datetime object to Pacific Time (PT)
    timestamp_pt = timestamp_utc_dt.astimezone(pt_timezone)
    # Format the timestamp in a human-readable way
    return timestamp_pt.strftime('%Y-%m-%d %I:%M:%S %p %Z')

In [33]:
import requests
import json

# Replace this with your Discord webhook URL
webhook_url = "https://discord.com/api/webhooks/1203794391362437242/9kOW437LYjip9F_FCUu30_-DnuZhqpY1t5TtnSCNtPTKw6KZRYG18dvM8KAvlH0rBohu"

def send_to_discord(message):
    # Create a dictionary with the message content
    payload = {
        "content": message
    }

    # Convert the payload to JSON
    payload_json = json.dumps(payload)

    # Send the POST request to the webhook URL
    response = requests.post(webhook_url, data=payload_json, headers={'Content-Type': 'application/json'})

In [40]:
import random

events = get_today_nba_events()
chosen_event = random.choice(events)
event_player_props = get_player_props(chosen_event["id"])
chosen_player_prop = random.choice(event_player_props)

message = f"""
## Today's Bet
### Game
{chosen_event['away_team']}@{chosen_event['home_team']} starting at {get_date_formatted_pt(chosen_event['commence_time'])}.
### Bet
{chosen_player_prop['name']} to get {chosen_player_prop['type']} {chosen_player_prop['point']} points. Price: {chosen_player_prop['price']}
"""
send_to_discord(message)